# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [15]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [16]:
# Load the data
file_path ='LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [17]:
# Create our features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_encoded = pd.get_dummies(df, columns=["hardship_flag", "debt_settlement_flag", "pymnt_plan", "verification_status","home_ownership"])
df_encoded['loan_status']=le.fit_transform(df_encoded['loan_status'])
df_encoded.head(30)

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,1,1,1,0,1,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,1,1,0,0,1,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,1,1,0,0,1,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1,1,1,0,0,1,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,1,1,1,1,0,0,0,1,0,0
5,25000.0,0.1797,634.43,90000.0,1,18.49,1.0,1.0,14.0,0.0,...,1,1,1,0,1,0,0,1,0,0
6,20400.0,0.2000,540.48,51315.0,1,25.63,0.0,2.0,8.0,1.0,...,1,1,1,0,1,0,0,0,0,1
7,10000.0,0.1557,349.46,66000.0,1,9.00,0.0,0.0,11.0,0.0,...,1,1,1,0,1,0,0,0,1,0
8,14000.0,0.0881,443.96,45000.0,1,10.24,0.0,1.0,9.0,0.0,...,1,1,1,1,0,0,0,0,0,1
9,10000.0,0.1474,345.39,63295.0,1,39.34,0.0,0.0,27.0,0.0,...,1,1,1,0,0,1,0,0,0,1


In [28]:
X = df.drop(columns='loan_status')
X=pd.get_dummies(X)
y=le.fit_transform(df['loan_status'])



In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)
counter=y_train

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [31]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [32]:
X_resampled

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10000.0,0.1131,328.86,49000.0,24.54,0.0,0.0,10.0,1.0,6580.0,...,0,1,0,1,0,1,1,0,1,1
1,21000.0,0.1072,684.74,57400.0,34.52,0.0,0.0,12.0,0.0,18688.0,...,0,1,0,1,0,1,1,0,1,1
2,35000.0,0.0881,1109.90,180000.0,3.01,0.0,2.0,7.0,0.0,836.0,...,0,1,0,1,0,1,1,0,1,1
3,20000.0,0.0819,628.49,80000.0,26.51,0.0,0.0,15.0,0.0,2615.0,...,0,1,0,1,0,1,1,0,1,1
4,12600.0,0.1356,427.96,71000.0,22.38,1.0,2.0,7.0,0.0,9073.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102699,36000.0,0.1557,867.25,61000.0,18.97,1.0,1.0,7.0,0.0,2939.0,...,1,1,0,1,0,1,1,0,1,1
102700,10800.0,0.1992,285.66,72000.0,23.83,0.0,4.0,14.0,0.0,5960.0,...,0,1,0,1,0,1,1,0,1,1
102701,14000.0,0.1447,329.18,68000.0,21.67,0.0,2.0,18.0,0.0,8191.0,...,0,1,0,1,0,1,1,0,1,1
102702,29250.0,0.1797,742.29,155000.0,8.41,1.0,0.0,8.0,0.0,1996.0,...,1,1,0,1,0,1,1,0,1,1


In [33]:
y_resampled

array([1, 1, 1, ..., 0, 0, 0])

In [34]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   55,    32],
       [ 6177, 10941]], dtype=int64)

In [36]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6356678390562868

In [37]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.63      0.64      0.02      0.64      0.40        87
          1       1.00      0.64      0.63      0.78      0.64      0.40     17118

avg / total       0.99      0.64      0.63      0.78      0.64      0.40     17205



### SMOTE Oversampling

In [38]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [39]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.65      0.02      0.62      0.38        87
          1       1.00      0.65      0.59      0.78      0.62      0.38     17118

avg / total       0.99      0.65      0.59      0.78      0.62      0.38     17205



In [40]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6165641329352849

In [41]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   51,    36],
       [ 6044, 11074]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.65      0.02      0.62      0.38        87
          1       1.00      0.65      0.59      0.78      0.62      0.38     17118

avg / total       0.99      0.65      0.59      0.78      0.62      0.38     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [48]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [49]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [50]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6165641329352849

In [52]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   51,    36],
       [ 6044, 11074]], dtype=int64)

In [53]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.65      0.02      0.62      0.38        87
          1       1.00      0.65      0.59      0.78      0.62      0.38     17118

avg / total       0.99      0.65      0.59      0.78      0.62      0.38     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [54]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [55]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [56]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  66,   21],
       [7124, 9994]], dtype=int64)

In [57]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6712252881620879

In [58]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.76      0.58      0.02      0.67      0.45        87
          1       1.00      0.58      0.76      0.74      0.67      0.44     17118

avg / total       0.99      0.58      0.76      0.73      0.67      0.44     17205

